In [13]:
import numpy as np
import pandas as pd
from log_analysis import get_crawl_status, load_dns_data_from_pcap_csvs

from os.path import isdir, join
from datetime import datetime
from glob import glob
from tld import get_fld

from collections import Counter
from itertools import chain

AMAZON_CRAWL = 'amazon-data-20190415-124534'
ROKU_CRAWL = 'roku-data-20190412-122224/'

ROOT_CRAWL_DIR = '/mnt/iot-house/crawl-data/'
if not isdir(ROOT_CRAWL_DIR):
    # ROOT_CRAWL_DIR = '/media/gacar/Data/iot-house/crawl-data/'
    ROOT_CRAWL_DIR = '/home/gacar/dev/smart-tv/data'

crawl_data_dir_amazon = join(ROOT_CRAWL_DIR, AMAZON_CRAWL)
crawl_data_dir_roku = join(ROOT_CRAWL_DIR, ROKU_CRAWL)

crawl_data_dirs = {
    "Amazon": crawl_data_dir_amazon,
    "Roku": crawl_data_dir_roku,
}

In [2]:
roku_crawl_results = get_crawl_status(crawl_data_dir_roku)
amazon_crawl_results = get_crawl_status(crawl_data_dir_amazon)
crawl_results = roku_crawl_results.copy()
crawl_results.update(amazon_crawl_results)

## Total number of channels crawled

In [3]:
print ("No. of channels crawled: Roku", len(roku_crawl_results))
print ("No. of channels crawled: Amazon", len(amazon_crawl_results))

No. of channels crawled: Roku 150
No. of channels crawled: Amazon 100


## Roku failed crawls

In [4]:
roku_failed_cnt = 0
for ch, result in roku_crawl_results.items():
    if result != "TERMINATED":
        # print (ch, result)
        roku_failed_cnt += 1
print ("%d of the %d channels failed" % (roku_failed_cnt, len(roku_crawl_results)))
counts = Counter(roku_crawl_results.values())
print (counts)

9 of the 150 channels failed
Counter({'TERMINATED': 141, 'INSTALLING': 8, 'LAUNCHING': 1})


## Amazon failed crawls

In [5]:
amazon_failed_cnt = 0
for ch, result in amazon_crawl_results.items():
    if result != "TERMINATED":
        # print (ch, result)
        amazon_failed_cnt += 1
print ("%d of the %d channels failed" % (amazon_failed_cnt, len(amazon_crawl_results)))
counts = Counter(amazon_crawl_results.values())
print (counts)


5 of the 100 channels failed
Counter({'TERMINATED': 95, 'LAUNCHING': 3, 'INSTALLING': 1, 'TERMINATING': 1})
